In [ ]:
## Install and import packages
!pip install openmeteo-requests
!pip install requests-cache retry-requests
import pandas as pd
import os

## Access Google Drive for dataset
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/deep learning project summer 2024/data'


ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.3 MB/s eta 0:00:00


ValueError: mount failed

In [ ]:
# path = '/content/drive/MyDrive/Colab Notebook/Colab Notebook/Erdos Institute Summer 2024/data-science-may-2024/data 2008 - 2023/'
#path = '/content/drive/MyDrive/Colab Notebook/Colab Notebook/Erdos Institute Summer 2024/data-science-may-2024/data 2008 - 2023'


In [ ]:
## Load reported dengue cases data downloaded and manually recorded from https://old.dghs.gov.bd/index.php/bd/home/5200-daily-dengue-status-report
file_case = '2008_2023_Final_BD_Whole_DengueAndClimateBangladesh.csv'
df_case = pd.read_csv(os.path.join(path, file_case)).rename(columns={'YEAR' : 'year', 'MONTH' : 'month', 'DENGUE IN' : 'case'})[['year','month','case']]

df_pop_case = df_case

In [ ]:
## Load weather data downloaded or retrieved from https://open-meteo.com/en/docs/historical-weather-api
# if os.path.isfile(os.path.join(path, 'weather.csv')): # Use downloaded data
#   df_weather = pd.read_csv(os.path.join(path, 'weather.csv'))
# else: # Download data
import openmeteo_requests

import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
"latitude": 24,
"longitude": 90,
"start_date": "2008-01-01",
"end_date": "2024-08-07",
"daily": ["temperature_2m_mean", "apparent_temperature_mean", "rain_sum"],
"timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(1).ValuesAsNumpy()
daily_rain_sum = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["rain_sum"] = daily_rain_sum

df_weather = pd.DataFrame(data = daily_data)
df_weather.to_csv(os.path.join(path, 'Bangladesh_daily_weather.csv'), sep=',', encoding='utf-8', index=False)
display(df_weather)

Coordinates 24.007028579711914°N 90.0°E
Elevation 11.0 m asl
Timezone b'Asia/Dhaka' b'+06'
Timezone difference to GMT+0 21600 s


,date,temperature_2m_mean,apparent_temperature_mean,rain_sum
0,2007-12-31 18:00:00+00:00,18.252081,18.071039,0.000000
1,2008-01-01 18:00:00+00:00,18.318748,18.713242,0.000000
2,2008-01-02 18:00:00+00:00,18.885416,19.297153,0.000000
3,2008-01-03 18:00:00+00:00,19.356251,19.554691,0.000000
4,2008-01-04 18:00:00+00:00,18.927084,18.437437,0.000000
...,...,...,...,...
6059,2024-08-02 18:00:00+00:00,26.752083,31.348358,27.000000
6060,2024-08-03 18:00:00+00:00,27.818750,32.845352,3.799999
6061,2024-08-04 18:00:00+00:00,28.479166,34.204639,8.099999
6062,2024-08-05 18:00:00+00:00,28.249998,33.458424,9.400000


In [ ]:
## Split year and month. Accumulate data for each month by averaging
df_weather['year'] = pd.DatetimeIndex(df_weather['date']).year
df_weather['month'] = pd.DatetimeIndex(df_weather['date']).month
df_weather = df_weather.drop('date', axis=1)
df_weather_grouped = df_weather.groupby(['year','month'], as_index=False).max() # this is max of each month in each year

In [ ]:
## Find data for common years and months
df = pd.merge(df_pop_case, df_weather_grouped, how='inner', on=['year','month'])

## Compute classification output variable: epidemic year from dengue cases and population
pop_reference = 1e6
thres_epidemic = 10
df['epidemic'] = df.apply(lambda row: 1 if row['case'] / row['pop'] * pop_reference >= thres_epidemic else 0, axis=1)
df.to_csv(os.path.join(path, 're_re_epidemic.csv'), sep=',', encoding='utf-8', index=False)

In [ ]:
df

,year,pop,month,case,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,rain,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,epidemic
0,2008,145421318.0,1,0,28.000000,99.359970,18.850000,28.940407,3.3,29.549999,23.650000,22.500000,24.700001,0.402,0.272,0.263,0.383,0
1,2008,145421318.0,2,0,29.250000,99.370224,19.600000,31.409039,2.7,31.900000,24.750000,22.299999,23.400000,0.314,0.273,0.238,0.368,0
2,2008,145421318.0,3,0,35.549999,99.100464,24.450001,39.063053,5.7,37.400002,30.000000,26.650000,23.500000,0.259,0.183,0.221,0.356,0
3,2008,145421318.0,4,0,37.299999,97.087570,25.950001,41.964233,8.8,40.650002,33.299999,29.350000,25.250000,0.439,0.219,0.194,0.342,0
4,2008,145421318.0,5,0,37.349998,98.238991,26.950001,43.002571,8.4,40.450001,33.750000,29.799999,27.000000,0.439,0.391,0.209,0.328,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,2023,172954319.0,8,71976,36.099998,100.000000,27.700001,43.051968,12.0,35.650002,32.099998,30.100000,28.850000,0.439,0.439,0.424,0.336,1
188,2023,172954319.0,9,79598,35.650002,99.111130,28.150000,43.996441,15.2,35.700001,32.049999,30.150000,29.150000,0.439,0.439,0.401,0.345,1
189,2023,172954319.0,10,67769,33.000000,100.000000,27.350000,40.530594,9.4,33.150002,30.100000,29.250000,29.150000,0.439,0.439,0.439,0.389,1
190,2023,172954319.0,11,40716,32.250000,98.422501,23.799999,36.977173,2.4,31.700001,28.049999,27.450001,28.700001,0.412,0.369,0.331,0.386,1


In [ ]:
## Split data into features+labels and train+validation+test
## year, month, pop, case, epidemic are not features
df = df.drop(columns=['soil_temperature_7_to_28cm',
       'soil_temperature_28_to_100cm', 'soil_temperature_100_to_255cm',
       'soil_moisture_0_to_7cm', 'soil_moisture_7_to_28cm',
       'soil_moisture_28_to_100cm', 'soil_moisture_100_to_255cm'])
df

,year,pop,month,case,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,rain,soil_temperature_0_to_7cm,epidemic
0,2008,145421318.0,1,0,28.000000,99.359970,18.850000,28.940407,3.3,29.549999,0
1,2008,145421318.0,2,0,29.250000,99.370224,19.600000,31.409039,2.7,31.900000,0
2,2008,145421318.0,3,0,35.549999,99.100464,24.450001,39.063053,5.7,37.400002,0
3,2008,145421318.0,4,0,37.299999,97.087570,25.950001,41.964233,8.8,40.650002,0
4,2008,145421318.0,5,0,37.349998,98.238991,26.950001,43.002571,8.4,40.450001,0
...,...,...,...,...,...,...,...,...,...,...,...
187,2023,172954319.0,8,71976,36.099998,100.000000,27.700001,43.051968,12.0,35.650002,1
188,2023,172954319.0,9,79598,35.650002,99.111130,28.150000,43.996441,15.2,35.700001,1
189,2023,172954319.0,10,67769,33.000000,100.000000,27.350000,40.530594,9.4,33.150002,1
190,2023,172954319.0,11,40716,32.250000,98.422501,23.799999,36.977173,2.4,31.700001,1


In [ ]:
data = df.drop(columns=['year', 'pop', 'month','epidemic'])
data

,case,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,rain,soil_temperature_0_to_7cm
0,0,28.000000,99.359970,18.850000,28.940407,3.3,29.549999
1,0,29.250000,99.370224,19.600000,31.409039,2.7,31.900000
2,0,35.549999,99.100464,24.450001,39.063053,5.7,37.400002
3,0,37.299999,97.087570,25.950001,41.964233,8.8,40.650002
4,0,37.349998,98.238991,26.950001,43.002571,8.4,40.450001
...,...,...,...,...,...,...,...
187,71976,36.099998,100.000000,27.700001,43.051968,12.0,35.650002
188,79598,35.650002,99.111130,28.150000,43.996441,15.2,35.700001
189,67769,33.000000,100.000000,27.350000,40.530594,9.4,33.150002
190,40716,32.250000,98.422501,23.799999,36.977173,2.4,31.700001
